# Step 1 User Query

In [1]:
import os
import subprocess

## Define alias and variables

In [2]:
# Parameters

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"

# Folder on local machine where to create the output and temporary folders
# output_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/output/"

# Location of the cache database for kypher
# cache_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/kypher"
# Whether to delete the cache database
# delete_database = False

# The names of files in the KGTK Wikidata distirbution that we will use in this notebook.
data_file_names = {
    "claims": "claims.tsv",
    "wiki_info": "wikidata_infobox.tsv",
    "p31": "P31.tsv",
    "p279star": "P279star.tsv",
    "labels": "labels.en.tsv"
}

# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

for key, value in data_file_names.items():
    variable = key.upper()
    os.environ[variable] = data_path + value
    kgtk_environment_variables.append(variable)

# output_file_names = {
#     "results": "industry_of_company.tsv",
#     "new_results": "new_industry_of_company.tsv",
#     "entity": "company.tsv",
#     "query_file": "company_wo_industry.tsv",
#     "direct_infer": "direct_infer.tsv",
#     "indirect_infer": "indirect_infer.tsv",
#     "infers": "infers.tsv",
#     "structured_literals": "structured_literals.tsv",
#     "nodes": "nodes.tsv",
#     "qnodes": "qnodes.tsv",
#     "correct_temp_1": "industry.type-constraints.instanceOf.correct_temp_1.tsv",
#     "correct_temp_2": "industry.type-constraints.instanceOf.correct_temp_2.tsv",
#     "incorrect_temp": "industry.type-constraints.instanceOf.incorrect_temp.tsv",
#     "correct": "industry.type-constraints.instanceOf.correct.tsv",
#     "incorrect": "industry.type-constraints.instanceOf.incorrect_temp.tsv"
# }

# os.environ['OUTPUT'] = output_path
# kgtk_environment_variables.append('OUTPUT')

# for key, value in output_file_names.items():
#     variable = key.upper()
#     os.environ[variable] = output_path + value
#     kgtk_environment_variables.append(variable)

# KGTK creates a SQLite database to index the knowledge graph.
# if cache_path:
#     os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
# else:
#     os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
# kgtk_environment_variables.append('STORE')
    
# Envronment variables with shortcuts to the commands we use often
# os.environ['kgtk'] = kgtk
# Use for debugging, but careful as it causes import to dataframes to break
# os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']
# os.environ['kypher'] = "kgtk query --graph-cache " + os.environ['STORE']
# kgtk_environment_variables.append('kgtk')
# kgtk_environment_variables.append('kypher')
    
# kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
CLAIMS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
WIKI_INFO: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/wikidata_infobox.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
LABELS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"


- property constraint (P2302)
- value-type constraint (Q21510865)

In [10]:
property_list = !kgtk query -i $CLAIMS --match 'c: (q)-[p:P2302]->(v:Q21510865)' --return 'q' --limit 100
property_list = property_list[1:5]

In [4]:
import re

In [61]:
property_dict = dict()
count = 0
with open(data_path + '/' + 'valueTypeConstraintValidator1.sh') as f:
    for line in f.readlines():
        property_matched = re.findall('claims.P[0-9]*.tsv', line)
        class_matched = re.findall('par in .{1,100}\] ', line)
        if property_matched and class_matched:
            proper = property_matched[0].strip('claims.').strip('.tsv')
            qnodes = class_matched[0].strip('par in ').strip()
            # print('%s --- %s--' % (proper, qnodes))
            property_dict[proper] = qnodes
            count += 1
            if count == 5:
                break

In [29]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

for prop in property_dict.keys():
    output_file_names = {
        "%s_results" % prop: "results.%s.tsv" % prop,
        "%s_new_results" % prop: "new.results.%s.tsv" % prop,
        "%s_entity" % prop: "entity.%s.tsv" % prop,
        "%s_query_file" % prop: "query.file.%s.tsv" % prop,
        "%s_direct_infer" % prop: "direct.infer.%s.tsv" % prop,
        "%s_indirect_infer" % prop: "indirect.infer.%s.tsv" % prop,
        "%s_infers" % prop: "infers.%s.tsv" % prop,
        "%s_structured_literals" % prop: "structured.literals.%s.tsv" % prop,
        "%s_nodes" % prop: "nodes.%s.tsv" % prop,
        "%s_qnodes" % prop: "qnodes.%s.tsv" % prop,
        "%s_correct_temp_1" % prop: "correct_temp_1.type-constraints.instanceOf.%s.tsv" % prop,
        "%s_correct_temp_2" % prop: "correct_temp_2.type-constraints.instanceOf.%s.tsv" % prop,
        "%s_incorrect_temp" % prop: "incorrect_temp.type-constraints.instanceOf.%s.tsv" % prop,
        "%s_correct" % prop: "correct.type-constraints.instanceOf.%s.tsv" % prop,
        "%s_incorrect" % prop: "incorrect_temp.type-constraints.instanceOf.%s.tsv" % prop
    }
    # print(output_file_names)
    for key, value in output_file_names.items():
        variable = key.upper()
        os.environ[variable] = output_path + value
        kgtk_environment_variables.append(variable)

# kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
CLAIMS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
WIKI_INFO: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/wikidata_infobox.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
LABELS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/"
P1000_RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/results.P1000.tsv"
P1000_NEW_RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/new.results.P1000.tsv"
P1000_ENTITY: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/entity.P1000.tsv"
P1000_QUERY_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/query.file.P1000.tsv"
P1000_DIRECT_INFER: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/direct.infer.P1000.tsv"
P1000_INDIRECT_INFER: "/nas/home/boh

P1136_CORRECT: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/correct.type-constraints.instanceOf.P1136.tsv"
P1136_INCORRECT: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/incorrect_temp.type-constraints.instanceOf.P1136.tsv"
P114_RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/results.P114.tsv"
P114_NEW_RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/new.results.P114.tsv"
P114_ENTITY: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/entity.P114.tsv"
P114_QUERY_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/query.file.P114.tsv"
P114_DIRECT_INFER: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/direct.infer.P114.tsv"
P114_INDIRECT_INFER: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/indirect.infer.P114.tsv"
P114_INFERS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/infers.P114.tsv"
P114_STRUCTURED_LITERALS: "/nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/structured.li

# Step 2: Wikidata Results

**TODO**: perform label query.

In [63]:
for prop, val in property_dict.items():
    command = "kgtk query -i $CLAIMS -i $P31 -i $P279STAR --match \'P31: (entity)-[]->(class), P279star: (class)-[]->(par), claims: (entity)-[p:%s]->(value)\' --where 'par in %s' --return \'entity as node1, p.label as label, value as node2\' -o $%s_RESULTS" % (prop, val, prop) 
    print(command)
    code = os.system(command)
    print(code)

kgtk query -i $CLAIMS -i $P31 -i $P279STAR --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par), claims: (entity)-[p:P1000]->(value)' --where 'par in ["Q1241356"]' --return 'entity as node1, p.label as label, value as node2' -o $P1000_RESULTS
0
kgtk query -i $CLAIMS -i $P31 -i $P279STAR --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par), claims: (entity)-[p:P1001]->(value)' --where 'par in ["Q20926517", "Q2881272", "Q2882257", "Q3624078", "Q3895768", "Q3918", "Q56061", "Q82794"]' --return 'entity as node1, p.label as label, value as node2' -o $P1001_RESULTS
0
kgtk query -i $CLAIMS -i $P31 -i $P279STAR --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par), claims: (entity)-[p:P1002]->(value)' --where 'par in ["Q2576663"]' --return 'entity as node1, p.label as label, value as node2' -o $P1002_RESULTS
0
kgtk query -i $CLAIMS -i $P31 -i $P279STAR --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par), claims: (entity)-[p:P1018]->(value)' --where '

In [65]:
!head $P102_RESULTS | column -ts $'\t'

node1      label  node2
Q12046465  P102   Q828099
Q12046465  P102   Q828099
Q12046466  P102   Q1586610
Q12046466  P102   Q1586610
Q12046466  P102   Q651129
Q12046466  P102   Q651129
Q12046467  P102   Q1784683
Q12046467  P102   Q1784683
Q15639867  P102   Q341148


## Count known results in Wikidata database:

In [74]:
results_lines = dict()
for prop in property_dict.keys():
    output = subprocess.check_output("wc -l < $%s_RESULTS" % prop, shell=True)
    output = output.decode("utf-8").strip()
    results_lines[prop] = int(output)
print(results_lines)

{'P1000': 1, 'P1001': 57645, 'P1002': 35, 'P1018': 1, 'P102': 164}


## Find unknow results in Wikidata database:

- Find all entities

**TODO**: What to put at the position of Q783794

In [93]:
for prop, val in property_dict.items():
    command = "kgtk query -i $P31 -i $P279STAR \
        --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par)' \
        --where 'par in %s' \
        --return 'entity as node1, \"P31\" as label, \"Q783794\" as node2' \
        -o $%s_ENTITY" % (val, prop)
    print(command)
    code = os.system(command)
    print(code)

kgtk query -i $P31 -i $P279STAR         --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par)'         --where 'par in ["Q1241356"]'         --return 'entity as node1, "P31" as label, "Q783794" as node2'         -o $P1000_ENTITY
0
kgtk query -i $P31 -i $P279STAR         --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par)'         --where 'par in ["Q20926517", "Q2881272", "Q2882257", "Q3624078", "Q3895768", "Q3918", "Q56061", "Q82794"]'         --return 'entity as node1, "P31" as label, "Q783794" as node2'         -o $P1001_ENTITY
0
kgtk query -i $P31 -i $P279STAR         --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par)'         --where 'par in ["Q2576663"]'         --return 'entity as node1, "P31" as label, "Q783794" as node2'         -o $P1002_ENTITY
0
kgtk query -i $P31 -i $P279STAR         --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(par)'         --where 'par in ["Q43229", "Q5"]'         --return 'entity as node1, "P31" as label, "

- Eliminate entities who have property / properties

In [94]:
for prop, val in property_dict.items():
    command = "kgtk ifnotexists -i $%s_ENTITY \
        --filter-on $%s_RESULTS \
        --input-keys node1 \
        --filter-keys node1 \
        -o $%s_QUERY_FILE" % (prop, prop, prop)
    print(command)
    code = os.system(command)
    print(code)

kgtk ifnotexists -i $P1000_ENTITY         --filter-on $P1000_RESULTS         --input-keys node1         --filter-keys node1         -o $P1000_QUERY_FILE
0
kgtk ifnotexists -i $P1001_ENTITY         --filter-on $P1001_RESULTS         --input-keys node1         --filter-keys node1         -o $P1001_QUERY_FILE
0
kgtk ifnotexists -i $P1002_ENTITY         --filter-on $P1002_RESULTS         --input-keys node1         --filter-keys node1         -o $P1002_QUERY_FILE
0
kgtk ifnotexists -i $P1018_ENTITY         --filter-on $P1018_RESULTS         --input-keys node1         --filter-keys node1         -o $P1018_QUERY_FILE
0
kgtk ifnotexists -i $P102_ENTITY         --filter-on $P102_RESULTS         --input-keys node1         --filter-keys node1         -o $P102_QUERY_FILE
0


## Count unknown results

In [95]:
for prop, val in property_dict.items():
    command = "kgtk query -i $%s_QUERY_FILE \
        --match '(p)-[]->()' \
        --return 'count(distinct p) as N'" % prop
    output = subprocess.check_output(command, shell=True)
    output = output.decode("utf-8").strip().split()
    print(output)

['N', '76']
['N', '5581513']
['N', '40']
['N', '12814146']
['N', '311558']


# Step 3 Selection of Additional KG(s)

# Step 4 Schema Alignment

## Entity resolution

Use query results from Wikidata database to infer properties in Wikidata infobox and return the most frequent property.

In [78]:
!kgtk query -i $P1001_RESULTS -i $WIKI_INFO \
            --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)' \
            --return 'entity, p.label, v as node2' \
            -o $P1001_DIRECT_INFER

In [79]:
!head $P1001_DIRECT_INFER | column -ts $'\t'

node1     label                  node2
Q1144565  property:country       Q34
Q1189704  property:membership    Q857330
Q1189704  property:membership    Q857330
Q1312762  property:country       Q34
Q154797   property:meetingPlace  Q183
Q165008   property:eventStart    Q469143
Q165008   property:eventStart    Q469143
Q165008   property:eventStart    Q469143
Q165008   property:eventStart    Q469143


**TODO**: Here we just use simple qnodes query first.

In [80]:
command = "kgtk query -i $P1001_RESULTS -i $WIKI_INFO \
           --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)' \
           --return 'entity, p.label, v as node2' \
           -o $P1001_DIRECT_INFER"
print(command)

kgtk query -i $P1001_RESULTS -i $WIKI_INFO            --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'            --return 'entity, p.label, v as node2'            -o $P1001_DIRECT_INFER


In [81]:
os.system(command)

0

In [82]:
for prop, val in property_dict.items():
    command = "kgtk query -i $%s_RESULTS -i $WIKI_INFO \
        --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)' \
        --return 'entity, p.label, v as node2' \
        -o $%s_DIRECT_INFER" % (prop, prop)
    print(command)
    code = os.system(command)
    print(code)

kgtk query -i $P1000_RESULTS -i $WIKI_INFO         --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'         --return 'entity, p.label, v as node2'         -o $P1000_DIRECT_INFER
0
kgtk query -i $P1001_RESULTS -i $WIKI_INFO         --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'         --return 'entity, p.label, v as node2'         -o $P1001_DIRECT_INFER
0
kgtk query -i $P1002_RESULTS -i $WIKI_INFO         --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'         --return 'entity, p.label, v as node2'         -o $P1002_DIRECT_INFER
0
kgtk query -i $P1018_RESULTS -i $WIKI_INFO         --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'         --return 'entity, p.label, v as node2'         -o $P1018_DIRECT_INFER
0
kgtk query -i $P102_RESULTS -i $WIKI_INFO         --match 'r: (entity)-[]->(v), w: (entity)-[p]->(v)'         --return 'entity, p.label, v as node2'         -o $P102_DIRECT_INFER
0


## Property mapping

In [87]:
!kgtk query -i $P1001_DIRECT_INFER \
            --match '(q)-[p]->(v)' \
            --return 'p.label, count(v) as N' \
            --order-by 'N desc' \
            --limit 1

label	N
property:seat	38


In [91]:
property_mapping = dict()
for prop, val in property_dict.items():
    command = "kgtk query -i $%s_DIRECT_INFER \
        --match '(q)-[p]->(v)' \
        --return 'p.label, count(v) as N' \
        --order-by 'N desc' \
        --limit 1" % prop
    output = subprocess.check_output(command, shell=True)
    output = output.decode("utf-8").strip().split()
    print(output)
    if len(output) == 4:
        property_mapping[prop] = output[2]

['label', 'N']
['label', 'N', 'property:seat', '38']
['label', 'N']
['label', 'N']
['label', 'N', 'property:parties', '14']


# Step 5 Results from other KG(s)

For those entities don't have property value, query in Wikidata infobox:

In [107]:
for prop, val in property_mapping.items():
    command = "kgtk query -i $%s_QUERY_FILE -i $WIKI_INFO \
        --match 'q: (entity)-[]->(), w: (entity)-[property]->(value)' \
        --where 'property.label = \"%s\"' \
        --return 'distinct entity, property.label, value' \
        -o $%s_NEW_RESULTS" % (prop, val, prop)
    print(command)
    code = os.system(command)
    print(code)

kgtk query -i $P1001_QUERY_FILE -i $WIKI_INFO         --match 'q: (entity)-[]->(), w: (entity)-[property]->(value)'         --where 'property.label = "property:seat"'         --return 'distinct entity, property.label, value'         -o $P1001_NEW_RESULTS
0
kgtk query -i $P102_QUERY_FILE -i $WIKI_INFO         --match 'q: (entity)-[]->(), w: (entity)-[property]->(value)'         --where 'property.label = "property:parties"'         --return 'distinct entity, property.label, value'         -o $P102_NEW_RESULTS
0


- Count rows of new findings:

In [108]:
for prop, val in property_mapping.items():
    output = subprocess.check_output("wc -l < $%s_NEW_RESULTS" % prop, shell=True)
    output = output.decode("utf-8").strip()
    print(output)

19016
72


- Count unique politicians of new findings:

In [110]:
for prop, val in property_mapping.items():
    command = "kgtk query -i $%s_NEW_RESULTS \
        --match 'n: (p)-[]->()' \
        --return 'count(distinct p) as N'" % prop
    output = subprocess.check_output(command, shell=True)
    output = output.decode("utf-8").strip().split()
    print(output[1])

18105
32


**TODO**: filter out those don't have new findings.

# Step 6 Datatype Filtering

## 1. Filter Structured literals:

In [112]:
for prop, val in property_mapping.items():
    command = "kgtk query -i $%s_NEW_RESULTS -i $WIKI_INFO \
        --match 'n: (q)-[p]->(s), w: (s)-[sv]->(v)' \
        --where 'NOT kgtk_lqstring(s) AND NOT kgtk_number(s) AND sv.label = \"dbpedia:structured_value\"' \
        --return 'q, p.label, s' \
        -o $%s_STRUCTURED_LITERALS" % (prop, prop)
    print(command)
    code = os.system(command)
    print(code)

kgtk query -i $P1001_NEW_RESULTS -i $WIKI_INFO         --match 'n: (q)-[p]->(s), w: (s)-[sv]->(v)'         --where 'NOT kgtk_lqstring(s) AND NOT kgtk_number(s) AND sv.label = "dbpedia:structured_value"'         --return 'q, p.label, s'         -o $P1001_STRUCTURED_LITERALS
0
kgtk query -i $P102_NEW_RESULTS -i $WIKI_INFO         --match 'n: (q)-[p]->(s), w: (s)-[sv]->(v)'         --where 'NOT kgtk_lqstring(s) AND NOT kgtk_number(s) AND sv.label = "dbpedia:structured_value"'         --return 'q, p.label, s'         -o $P102_STRUCTURED_LITERALS
0


In [116]:
!head $P102_STRUCTURED_LITERALS | column -ts $'\t'

node1      label             node2
Q50828855  property:parties  nodemxZbyK2VRrGoaxfdLmyLxw-1906669


In [117]:
!wc -l $P102_STRUCTURED_LITERALS

2 /nas/home/bohuizha/KG/hunger-for-knowledge/batch_output/structured.literals.P102.tsv


## 2. Filter Qnodes

In [114]:
for prop, val in property_mapping.items():
    command_1 = "kgtk query -i $%s_NEW_RESULTS \
        --match 'n: (q)-[p]->(v)' \
        --where 'NOT kgtk_lqstring(v) AND NOT kgtk_number(v)' \
        --return 'distinct q, p.label, v' \
        -o $%s_NODES" % (prop, prop)
    print(command_1)
    code_1 = os.system(command_1)
    print(code_1)
    command_2 = "kgtk ifnotexists -i $%s_NODES \
        --filter-on $%s_STRUCTURED_LITERALS \
        -o $%s_QNODES" % (prop, prop, prop)
    print(command_2)
    code_2 = os.system(command_2)
    print(code_2)

kgtk query -i $P1001_NEW_RESULTS         --match 'n: (q)-[p]->(v)'         --where 'NOT kgtk_lqstring(v) AND NOT kgtk_number(v)'         --return 'distinct q, p.label, v'         -o $P1001_NODES
0
kgtk ifnotexists -i $P1001_NODES         --filter-on $P1001_STRUCTURED_LITERALS         -o $P1001_QNODES
0
kgtk query -i $P102_NEW_RESULTS         --match 'n: (q)-[p]->(v)'         --where 'NOT kgtk_lqstring(v) AND NOT kgtk_number(v)'         --return 'distinct q, p.label, v'         -o $P102_NODES
0
kgtk ifnotexists -i $P102_NODES         --filter-on $P102_STRUCTURED_LITERALS         -o $P102_QNODES
0


In [115]:
for prop, val in property_mapping.items():
    output = subprocess.check_output("wc -l < $%s_QNODES" % prop, shell=True)
    output = output.decode("utf-8").strip()
    print(output)

15981
49


**TODO**: filter out those don't have qnodes.

# Step 7 Quality Checking

In [120]:
for prop, val in property_mapping.items():
    command_1 = "kgtk query -i $%s_QNODES $P31 $P279STAR \
        --match 'q: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(nodex), P279star: (nodex)-[]->(par)' \
        --where 'par in %s' \
        --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
        -o $%s_CORRECT_TEMP_1" % (prop, property_dict[prop], prop)
    print(command_1)
    code_1 = os.system(command_1)
    print(code_1)
    command_2 = "kgtk ifnotexists -i $%s_QNODES \
        --filter-on $%s_CORRECT_TEMP_1 \
        --input-keys node1 node2 \
        --filter-keys node1 node2 \
        -o $%s_INCORRECT_TEMP" % (prop, prop, prop)
    print(command_2)
    code_2 = os.system(command_2)
    print(code_2)
    command_3 = "kgtk query -i $%s_INCORRECT_TEMP $P279STAR \
        --match 'i: (node1)-[nodeProp]->(node2), P279star: (node2)-[]->(par)' \
        --where 'par in %s' \
        --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
        -o $%s_CORRECT_TEMP_2" % (prop, property_dict[prop], prop)
    print(command_3)
    code_3 = os.system(command_3)
    print(code_3)
    command_4 = "kgtk ifnotexists -i $%s_INCORRECT_TEMP \
        --filter-on $%s_CORRECT_TEMP_2 \
        --input-keys node1 node2 \
        --filter-keys node1 node2 \
        -o $%s_INCORRECT" % (prop, prop, prop)
    print(command_4)
    code_4 = os.system(command_4)
    print(code_4)
    command_5 = "kgtk cat -i $%s_CORRECT_TEMP_1 $%s_CORRECT_TEMP_2 \
        -o $%s_CORRECT" % (prop, prop, prop)
    print(command_5)
    code_5 = os.system(command_5)
    print(code_5)

kgtk query -i $P1001_QNODES $P31 $P279STAR         --match 'q: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(nodex), P279star: (nodex)-[]->(par)'         --where 'par in ["Q20926517", "Q2881272", "Q2882257", "Q3624078", "Q3895768", "Q3918", "Q56061", "Q82794"]'         --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`'         -o $P1001_CORRECT_TEMP_1
0
kgtk ifnotexists -i $P1001_QNODES         --filter-on $P1001_CORRECT_TEMP_1         --input-keys node1 node2         --filter-keys node1 node2         -o $P1001_INCORRECT_TEMP
0
kgtk query -i $P1001_INCORRECT_TEMP $P279STAR         --match 'i: (node1)-[nodeProp]->(node2), P279star: (node2)-[]->(par)'         --where 'par in ["Q20926517", "Q2881272", "Q2882257", "Q3624078", "Q3895768", "Q3918", "Q56061", "Q82794"]'         --return 'distinct node1 as `node1`, nodeProp.label as `label`, node2 as `node2`'         -o $P1001_CORRECT_TEMP_2
0
kgtk ifnotexists -i $P1001_INCORRECT_TEMP         --filter-on $P1001_

In [122]:
for prop, val in property_mapping.items():
    output_1 = subprocess.check_output("wc -l < $%s_CORRECT" % prop, shell=True)
    output_1 = output_1.decode("utf-8").strip()
    correct_lines = int(output_1) - 1
    output_2 = subprocess.check_output("wc -l < $%s_INCORRECT" % prop, shell=True)
    output_2 = output_2.decode("utf-8").strip()
    incorrect_lines = int(output_2) - 1
    output_3 = subprocess.check_output("wc -l < $%s_QNODES" % prop, shell=True)
    output_3 = output_3.decode("utf-8").strip()
    qnode_lines = int(output_3) - 1
    print(correct_lines, incorrect_lines, qnode_lines)
    assert correct_lines + incorrect_lines == qnode_lines, "The sum is not correct!"

15747 233 15980
48 0 48
